In [54]:
import json
tree_data = {
    "author": "John Doe",
    "version": "1.0",
    "mapping": {
        "node1": {
            "parent": None,
            "data": "Some data for node1"
        },
        "node2": {
            "parent": "node1",
            "data": "Some data for node2"
        },
        "node3": {
            "parent": "node1",
            "data": "Some data for node3"
        },
        "node4": {
            "parent": "node3",
            "data": "Some data for node4"
        },
        "node5": {
            "parent": "node3",
            "data": "Some data for node5"
        }
    }
}
#print(json.dumps(tree_data, indent=2))

In [55]:
import treekit as tk

# load a tree from tree_data
tree = tk.DictTree.from_dict(data=tree_data, mapping_key="mapping")
print(tree)

node1
|-- node2
+-- node3
    |-- node4
    +-- node5


We show that it's easy to regenerate any JSON files that may have been used
to generate the DictTree 'tree'. So, JSON is a good format for storing and
transmitting the DictTree 'tree'.

In [56]:
json.dumps(tree.to_dict(),  indent=2) == json.dumps(tree_data, indent=2)

True

In [57]:
tree.leaves()

[Node('/node1/node2', payload={'parent': 'node1', 'data': 'Some data for node2'}),
 Node('/node1/node3/node4', payload={'parent': 'node3', 'data': 'Some data for node4'}),
 Node('/node1/node3/node5', payload={'parent': 'node3', 'data': 'Some data for node5'})]

In [58]:
child = tree.get_node('node3').children
print(child)
print(child[0].name)
print(child[0].parent.name)
print(child[0].ancestors)
print(child[0].descendants)
print(child[0].height)
print(child[0].path)
print(child[0].siblings)
print(child[0].depth)


(Node('/node1/node3/node4', payload={'parent': 'node3', 'data': 'Some data for node4'}), Node('/node1/node3/node5', payload={'parent': 'node3', 'data': 'Some data for node5'}))
node4
node3
(Node('/node1', payload={'parent': None, 'data': 'Some data for node1'}), Node('/node1/node3', payload={'parent': 'node1', 'data': 'Some data for node3'}))
()
0
(Node('/node1', payload={'parent': None, 'data': 'Some data for node1'}), Node('/node1/node3', payload={'parent': 'node1', 'data': 'Some data for node3'}), Node('/node1/node3/node4', payload={'parent': 'node3', 'data': 'Some data for node4'}))
(Node('/node1/node3/node5', payload={'parent': 'node3', 'data': 'Some data for node5'}),)
2


In [59]:
tree.save("tree-test.png")

Here is the `tree-test.png` PNG.

![tree-test.png](tree-test.png)

We can show the different paths in the tree with:

In [60]:
print(tree.flatten(node_name=lambda n: n.name))

[['node1', 'node2'], ['node1', 'node3', 'node4'], ['node1', 'node3', 'node5']]


In [61]:
"node1" in tree

True

In [62]:
print(tree.get_node("node4").ancestors)
print(tree.root_node.descendants)
print(tree.height())

(Node('/node1', payload={'parent': None, 'data': 'Some data for node1'}), Node('/node1/node3', payload={'parent': 'node1', 'data': 'Some data for node3'}))
(Node('/node1/node2', payload={'parent': 'node1', 'data': 'Some data for node2'}), Node('/node1/node3', payload={'parent': 'node1', 'data': 'Some data for node3'}), Node('/node1/node3/node4', payload={'parent': 'node3', 'data': 'Some data for node4'}), Node('/node1/node3/node5', payload={'parent': 'node3', 'data': 'Some data for node5'}))
2


In [63]:
print(tree.get_node("node1").is_leaf)
print(tree.get_node("node2").is_root)
print(tree.get_node("node3").children)
print(tree.get_node("node4").depth)
print(tree.get_node("node4").parent)
print(tree.get_node("node2").siblings)

False
False
(Node('/node1/node3/node4', payload={'parent': 'node3', 'data': 'Some data for node4'}), Node('/node1/node3/node5', payload={'parent': 'node3', 'data': 'Some data for node5'}))
2
Node('/node1/node3', payload={'parent': 'node1', 'data': 'Some data for node3'})
(Node('/node1/node3', payload={'parent': 'node1', 'data': 'Some data for node3'}),)


In [64]:
print(tk.DictTree.spec())

# DictTree Specification

The dict (or JSON) data should have the following structure:

```json
{
    // Meta-data (optional key-value pairs)
    "<key>": "<value>", ...

    "<mapping_key>": {
        "<node_key>": {
            // Parent node key (optional)
            "parent": "<parent_node_key>",

            // Children node keys (optional)
            "children": [
                "<child_node_key>", ...
            ],
                
            // Node data (optional key-value pairs)
            "<key>": "<value>", ...
        },

        // More node key-value pairs
        ...
    }
}
```

where:

- <mapping_key> is the key in the JSON that maps to the structure of the tree. Default is "mapping".
- <node_key> is the key of the node.
- <parent_node_key> is the key of the parent node.
- <child_node_key> is the key of a child node.
- Additional key-value pairs can be added to each node to store arbitrary data.
- The JSON data can have additional key-value pairs for meta-data, 

In [65]:
nodes = tk.find_nodes(
    tree=tree,
    query="data=='Some data for node4' || node_key=='node1'",
    matcher=tk.matchers('bool'))
print(nodes)


[Node('/node1/node3/node4', payload={'parent': 'node3', 'data': 'Some data for node4'})]


In [66]:
tk.decorate_nodes(
    tree=tree,
    decs=tk.decorators(['depth', 'siblings', 'num_children', 'node_key']))
print(tree.to_string(node_name=lambda n: n.payload))

{'parent': None, 'data': 'Some data for node1', 'node_depth': '0', 'siblings': [], 'num_children': '2', 'node_key': 'node1'}
├── {'parent': 'node1', 'data': 'Some data for node2', 'node_depth': '1', 'siblings': ['node3'], 'num_children': '0', 'node_key': 'node2'}
└── {'parent': 'node1', 'data': 'Some data for node3', 'node_depth': '1', 'siblings': ['node2'], 'num_children': '2', 'node_key': 'node3'}
    ├── {'parent': 'node3', 'data': 'Some data for node4', 'node_depth': '2', 'siblings': ['node5'], 'num_children': '0', 'node_key': 'node4'}
    └── {'parent': 'node3', 'data': 'Some data for node5', 'node_depth': '2', 'siblings': ['node4'], 'num_children': '0', 'node_key': 'node5'}


In [89]:
nodes = tk.find_nodes(
    tree=tree,
    query="node_depth=='2' || node_key=='node1'",
    matcher=tk.matchers('bool'))
for n in nodes:
    print(n.name)

node1
node4
node5


In [68]:
nodes = tk.find_nodes(
    tree=tree,
    query="for node4",                  
    matcher=tk.matchers("regex"))
print(nodes)

[Node('/node1/node3/node4', payload={'parent': 'node3', 'data': 'Some data for node4', 'node_depth': '2', 'siblings': ['node5'], 'num_children': '0', 'node_key': 'node4'})]


In [69]:
nodes = tk.find_nodes(
    tree=tree,
    query=lambda n: n['data'] == 'Some data for node4',
    matcher=tk.matchers("pred"))
print(nodes)

[Node('/node1/node3/node4', payload={'parent': 'node3', 'data': 'Some data for node4', 'node_depth': '2', 'siblings': ['node5'], 'num_children': '0', 'node_key': 'node4'})]


In [70]:
import treekit as tk
tree_data_alt = {
    "author": "John Doe",
    "version": "2.0",
    "mapping": {
        "node1b": {
            "children": ["node2b", "node3b"],
            "data": "Some data for node1b"
        },
        "node2b": {
            "data": "Some data for node2b"
        },
        "node3b": {
            "data": "Some data for node3b"
        },
        "node4b": {
            "data": "Some data for node4b"
        },
    }
}
tree_alt = tk.DictTree.from_dict(tree_data_alt)
print(tree_alt) 

tree_alt.get_node("node4b").parent = tree_alt.get_node("node3b")

print(tree_alt)

node1b
|-- node2b
+-- node3b
node1b
|-- node2b
+-- node3b
    +-- node4b


In [71]:
new_node = tree_alt.add_node("node5b", parent_key="node3b", payload="Some data for node5b")


In [72]:
print(tree_alt)

node1b
|-- node2b
+-- node3b
    |-- node4b
    +-- node5b


In [73]:
new_tree = tk.DictTree.from_anytree(new_node)
new_tree.add_node("A", parent_key="node5b", payload="Some data for A")
new_tree.edit_node("node5b", payload="Some data for node5b edited")
print(new_tree.to_string(node_name=lambda n: n.payload))
print(tree_alt.to_string(node_name=lambda n: n.payload))


Some data for node5b edited
└── Some data for A
{'children': ['node2b', 'node3b'], 'data': 'Some data for node1b'}
├── {'data': 'Some data for node2b'}
└── {'data': 'Some data for node3b'}
    ├── {'data': 'Some data for node4b'}
    └── Some data for node5b


In [74]:
tree_alt.remove_node("node3b")
print(tree_alt)


node1b
+-- node2b


In [75]:
try:
    tree_alt.add_node("node15b", parent_key="node3", payload="Some data for node15b")
except Exception as e:
    print(e)
    tree_alt.add_node("node15b", payload="Some data for node15b")

Parent node node3 not found


In [76]:
print(tree_alt.get_node("node15b"))
print(tree_alt)

Node('/node15b', payload='Some data for node15b')
node1b
+-- node2b


In [77]:
print(tree_alt.is_connected())
tree_alt.remove_node("node15b")
print(tree_alt.is_connected())

False
True


In [78]:
tree_alt.edit_node("node1b", payload={"data": "Some different data for node1b"})
print(tree_alt.to_string(node_name = lambda n: f"{n.name} => {n.payload}"))


node1b => {'data': 'Some different data for node1b'}
└── node2b => {'data': 'Some data for node2b'}


In [79]:
tree.merge_under("node2", tree_alt)
new_tree.merge_under("A", tree)
print(tree)
print(new_tree)

node1
|-- node2
|   +-- node1b
|       +-- node2b
+-- node3
    |-- node4
    +-- node5
node5b
+-- A
    +-- node1
        |-- node2
        |   +-- node1b
        |       +-- node2b
        +-- node3
            |-- node4
            +-- node5


In [80]:
import logging
logging.basicConfig(level=logging.DEBUG)
tree.verify_integrity()

In [81]:
node2 = tree.remove_node("node2")

In [82]:
subtree = tk.DictTree.from_anytree(node2)

In [83]:
subtree.add_node("B", parent_key="node2", payload="Some data for B")
print(subtree)

node2
|-- node1b
|   +-- node2b
+-- B


In [84]:
print(tree)

node1
+-- node3
    |-- node4
    +-- node5


In [86]:
subtree.edit_node("node2", new_key="C")
print(subtree)

C
|-- node1b
|   +-- node2b
+-- B


In [92]:
print(new_tree)

nodes = tk.find_nodes(
    tree=new_tree,
    query="node_key=='node2'",
    matcher=tk.matchers("bool"))
print(tk.DictTree.from_anytree(nodes[0]))

node5b
+-- A
    +-- node1
        |-- node2
        |   +-- node1b
        |       +-- node2b
        +-- node3
            |-- node4
            +-- node5


AttributeError: 'list' object has no attribute 'name'